## Minimum Spaning Tree (MST)

In [2]:
import numpy as np
import pandas as pd
import os
import igraph as ig
import networkx as nx
import random
from gurobipy import Model,GRB,LinExpr,quicksum
import cartopy
import cartopy.crs as ccrs
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patheffects as PathEffects
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import itertools
from pyomo.environ import *

V = {'TX':1,
     'AK':2,
     'LA':3,
     'TN':4,
     'AL':5,
     'NC':6}

E = {('TX','AK'):446,
     ('TX','LA'):432,
     ('AK','LA'):343,
     ('AK','TN'):347,
     ('AK','AL'):463,
     ('LA','TN'):587,
     ('LA','AL'):365,
     ('TN','NC'):539,
     ('AL','TN'):281,
     ('AL','NC'):569}

S = list(itertools.combinations(V.keys(),3))+\
    list(itertools.combinations(V.keys(),4))+\
    list(itertools.combinations(V.keys(),5))
    
E_s = {s:[e for e in E if (e[0] in s and e[1] in s)] for s in S}

In [3]:
model = ConcreteModel()

# Define sets
model.vertices = Set(initialize=V.keys())
model.edges    = Set(initialize=E.keys())

# Define parameters
model.distance = Param(model.edges, initialize=E, within=NonNegativeReals)

# Define the decision variables
model.x = Var(model.edges, within=Binary)

# Define the objective function
model.obj = Objective(expr=sum(model.distance[(u, v)] * model.x[(u, v)] for (u, v) in model.edges), sense=minimize)

# Define the constraints

model.number_edges = ConstraintList()
model.number_edges.add(sum(model.x[(v, w)] for (v,w) in model.edges)==len(V)-1)

model.subtour_elim = ConstraintList()
for s in S:
    model.subtour_elim.add(sum(model.x[(v, w)] for (v,w) in model.edges 
                           if (v,w) in E_s[s])<=len(s)-1)
    



# Initialize the variables
for (v, w) in model.edges:
    model.x[(v, w)].value = 0

In [4]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model)

# Print the results
print('Overall distance of the minimum spanning tree:', model.obj())
for (v, w) in model.edges:
    print(f'Edge {v} - {w}: {model.x[(v, w)].value}')

Overall distance of the minimum spanning tree: 1942.0
Edge TX - AK: -0.0
Edge TX - LA: 1.0
Edge AK - LA: 1.0
Edge AK - TN: 1.0
Edge AK - AL: -0.0
Edge LA - TN: -0.0
Edge LA - AL: 0.0
Edge TN - NC: 1.0
Edge AL - TN: 1.0
Edge AL - NC: -0.0
